In [ ]:
#packages

import nilmtk
import pandas as pd
import numpy as np
import matplotlib.font_manager
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
import os, ast
from sys import argv
from time import time
import nilmtk
import nilmtk_contrib
from nilmtk.api import API
from nilmtk.disaggregate import Mean, FHMMExact, CO, Hart85
from nilmtk_contrib.disaggregate import AFHMM, AFHMM_SAC, DSC, DAE, Seq2Point, Seq2Seq, RNN, WindowGRU
nilmtk.Appliance.allow_synonyms=False
import time
import scipy.stats 
from datetime import datetime, timedelta
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import seaborn as sns
import math
from math import pi
import pickle

#load REFIT dataset
dataset = nilmtk.DataSet('REFIT.h5')

In [ ]:
BUILDING = 19

#sample rate
SAMPLE_RATE = '60s'

#experiment window
START = "2014-06-21"

END = "2015-06-20"

dataset.set_window(START, END)

# elec[1] refers to the mains
dataset_mains = next(dataset.buildings[BUILDING].elec[1].load())['power'].resample(SAMPLE_RATE).fillna("pad").values.tolist()
dataset_mains.remove(dataset_mains[0]) 

#function that computes total noise in the building aggregate
def get_noise(mains):
    noise_seq2 = mains
    for j in range(1, len(dataset.buildings[BUILDING].elec.appliances)):
        if dataset.buildings[BUILDING].elec[j+1].appliances[0].type['type'] != 'unknown':
            dataset_aux = next(dataset.buildings[BUILDING].elec[j+1].load())['power'].resample(SAMPLE_RATE).fillna("pad").values.tolist()                
            dataset_aux.remove(dataset_aux[0])
            
            if len(mains) != len(dataset_aux):
                print("length of mains observed sequence differs from that of the selected appliance")
            else:

                for i in range(0, len(mains)): 
                    noise_seq2[i][0] = noise_seq2[i][0] - dataset_aux[i][0]
            
    return noise_seq2
 
noise_total = get_noise(dataset_mains)
noise_total = [item for sublist in noise_total for item in sublist]
dataset_mains=next(dataset.buildings[BUILDING].elec[1].load())['power'].resample(SAMPLE_RATE).fillna("pad").values.tolist()
dataset_mains.remove(dataset_mains[0])

#get the proportion of noise in the building aggregate
def get_noise_prop_agg(mains, noise):
    noise_t = 0
    agg_total = 0    
    for i in range(0, len(noise)):
        noise_t += abs(noise[i])
        agg_total += abs(mains[i][0])
    return agg_total/noise_t


#get the appliance to noise ratio
def get_noise_prop_app(app, noise):
    noise_t = 0
    app_total = 0
    for i in range(0, len(noise)):
        app_total += abs(app[i][0])
        noise_t += abs(noise[i])
    return app_total/noise_t 

In [ ]:
#creation of dictionaries

#for saving the total noise for each appliance across the houses
noise_wm_60 = {}
noise_dw_60 = {}
noise_mw_60 = {}
noise_kt_60 = {}

#proportion of noise in the aggregate, and the appliance-to-noise ratios for each appliance
ratios_60 = {} #

In [ ]:
#appliances considered
load1 = "washing machine"

load2 = "dish washer"

load3 = "microwave"

load4 = "kettle"

#get power values sampled at 1/60hz
dataset_wm=next(dataset.buildings[BUILDING].elec[load1].load())['power'].resample(SAMPLE_RATE).fillna("pad").values.tolist()
dataset_wm.remove(dataset_wm[0])

dataset_dw=next(dataset.buildings[BUILDING].elec[load2].load())['power'].resample(SAMPLE_RATE).fillna("pad").values.tolist()
dataset_dw.remove(dataset_dw[0])

dataset_mw=next(dataset.buildings[BUILDING].elec[load3].load())['power'].resample(SAMPLE_RATE).fillna("pad").values.tolist()
dataset_mw.remove(dataset_mw[0])

dataset_kt=next(dataset.buildings[BUILDING].elec[load4].load())['power'].resample(SAMPLE_RATE).fillna("pad").values.tolist()
dataset_kt.remove(dataset_kt[0])

#The 20 REFIT houses ID ranges from 1 to 21, except 14, but they are indexed from 1 to 20 
#therefore for houses indexed above 14, BUILDING ID= index+1  
if BUILDING >= 14:
    BUILDING = BUILDING+1
    noise_wm_60[str(BUILDING)] = noise_total
else:
    noise_wm_60[str(BUILDING)] = noise_total
    
#then compute for the other appliances    


ratios_60[str(BUILDING)] = [get_noise_prop_agg(dataset_mains, noise_total), get_noise_prop_app(dataset_wm, noise_total),  
                             get_noise_prop_app(dataset_dw, noise_total), get_noise_prop_app(dataset_mw, noise_total)
                            get_noise_prop_app(dataset_kt, noise_total)]

In [ ]:
#save as pickle 
dbfile = open('ratios_60', 'ab')
b=pickle.dump(ratios_60, dbfile)

In [ ]:
#load pickle
ratios_60_pickle = open('ratios_60', 'rb')     
ratios_60_pickle = pickle.load(ratios_60_pickle)

ratios_60_pickle

In [ ]:
#get number of activations of all appliances in the building
#only the activation times larger than 1 minute are selected for calculating this number 

def get_total_activations(BUILDING, min_on):
    res = 0
    for j in range(1, len(dataset.buildings[BUILDING].elec.appliances)):
        if dataset.buildings[BUILDING].elec[j+1].appliances[0].type['type'] != 'unknown':
            res += len(dataset.buildings[BUILDING].elec[j+1].get_activations(min_on_duration = min_on))  
        
    return res    

# get number of activations of an appliance 
def get_load_activations (BUILDING, load, min_on):
    res = 0
    if dataset.buildings[BUILDING].elec[load].appliances[0].type['type'] != 'unknown':
        res = len(dataset.buildings[BUILDING].elec[load].get_activations(min_on_duration = min_on))      
    return res

In [ ]:
Houses = ['2' , '3' , '5' , '6' , '9' , '11' , '13' , '20'] #11 houses
#these houses are considered because they include at least one of each appliance: washing machine, dish washer, microwave 
#and kettle

LOAD = "washing machine"
res = []
for i in Houses:
    if int(i) < 14:
        res.append( (get_load_activations(int(i), LOAD, SAMPLE_RATE)/get_total_activations(int(i), SAMPLE_RATE))*ratios_60_pickle[i][0] )
    else: 
        res.append( (get_load_activations(int(i)-1, LOAD, SAMPLE_RATE)/get_total_activations(int(i)-1, SAMPLE_RATE))*ratios_60_pickle[i][0] )

In [ ]:
#compute signal-to-noise ratio
def SNR(data):
    res = []
    for i in Houses:
        res.append( 10*math.log10(data[i][0]) )
    return res

SNR(ratios_60_pickle)

In [ ]:
#compute noise-to-aggregate ratio
def NAR(data):
    res = []
    for i in Houses:
        res.append( 1/data[i][0] )
    return res

NAR(ratios_60_pickle)

In [ ]:
#illustration figure showed in the paper

labels = ["", "2", "3",  "5", "6", "9", "11", "13", "20", "9", "", "11", "", "13", "", "20", "" ]
anr = [0.06, 0.05, 0.04, 0.06, 0.07, 0.16, 0.03, 0.06]
aka = [0.221, 0.102, 0.102, 0.137, 0.145, 0.230, 0.174, 0.088]
wanr = [0.014, 0.005, 0.004, 0.009, 0.010, 0.038, 0.005, 0.005]

x = np.arange(8) 
width = 0.2

plt.rcParams["figure.figsize"] = (15,5)

fig, ax = plt.subplots()
rects1 = ax.bar(x - width, anr, width, label='ANR')
rects2 = ax.bar(x, aka, width, label='A_k/A')
rects3 = ax.bar(x + width, wanr, width, label='WANR')

ax.set_ylabel('Scores', fontsize = 16)
ax.set_xlabel('Houses', fontsize = 16)
ax.set_title('ANR, A_k/A and WANR scores across the selected houses', fontsize = 16)
ax.set_xticks(x, labels)
ax.set_yticks(np.arange(0, 0.27, 0.03))
ax.set_yticklabels(np.arange(0, 0.27, 0.03), fontsize = 16)
ax.set_xticklabels(labels, fontsize = 16)
ax.legend(fontsize = 16)

ax.grid(linestyle = '--', linewidth = 1)


fig.tight_layout()

plt.show()

In [ ]:
#boxplots of power consumption when the appliance is turned on

load = "microwave"

Houses = ['2', '3', '5', '6', '9', '10', '11', '13', '15', '18', '20']

KETTLE_dict = {}
for i in Houses:
    if int(i)<14:
        KETTLE = next(dataset.buildings[int(i)].elec[load].load(sample_period = 60)).values
    else:     
        KETTLE = next(dataset.buildings[int(i)-1].elec[load].load(sample_period = 60)).values    
    KETTLE = [KETTLE[j][0] for j in range(KETTLE.shape[0]) if KETTLE[j][0]>5]
    KETTLE_dict[i] = KETTLE

plt.rcParams["figure.figsize"] = (10, 5)
labels, data = KETTLE_dict.keys(), KETTLE_dict.values()
plt.boxplot(data)
plt.xticks(range(1, len(labels) + 1), labels, fontsize = 14)
plt.yticks(fontsize = 14)
plt.grid(linestyle = '--', linewidth = 1)
plt.title("Distribution of "+str(load)+" power consumption at 1/60 Hz when turned ON", fontsize = 16)
plt.xlabel("Houses", fontsize = 16)
plt.ylabel("Active Power (W)", fontsize = 16)
plt.show()

In [ ]:
house=2
dataset_wm=next(dataset.buildings[house].elec["washing machine"].load())['2014-06-21' : '2015-06-20']['power']

X = dataset_wm.values
tscv = TimeSeriesSplit(n_splits = 2)
print(tscv)
train_index_num = []
test_index_num = [] 
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index, :], X[test_index, :]
    train_index_num.append(train_index)
    test_index_num.append(test_index)
    
train_inic = dataset_wm[0: train_index_num[1][-1]].index.strftime('%Y-%m-%d')[0]
train_fim = dataset_wm[0: train_index_num[1][-1]].index.strftime('%Y-%m-%d')[-1]
test_inic = dataset_wm[0: test_index_num[1][-1]].index.strftime('%Y-%m-%d')[0]
test_fim = dataset_wm[0: test_index_num[1][-1]].index.strftime('%Y-%m-%d')[-1]    

In [ ]:
sampling_interval = 60
load = 'washing machine'

BUILDING = 5

train_inicio=train_inic
train_final=train_fim
test_inicio=test_inic
test_final= test_fim

experiment = {
    'power': { 'mains': ['active'], 'appliance': ['active'] },
    'sample_rate': sampling_interval,
    'appliances': [load],
    'methods': { 
        #'FHMMExact': FHMMExact({}) },
        #'CO': CO({})},
        #'DSC':DSC({'learning_rate':1e-11, 'iterations':50}) },
        #'RNN': RNN({'n_epochs': 50, 'batch_size': 1024})  },
        'DAE': DAE({'n_epochs': 50, 'batch_size': 1024}),
        'Seq2Point': Seq2Point({'n_epochs': epochs, 'batch_size': 128}),
        'Seq2Seq': Seq2Seq({'n_epochs': epochs, 'batch_size': 1024})}
        #'WindowGRU':WindowGRU({'n_epochs':epochs,'batch_size':1024}),
    'train': {
        'datasets': {
            'REFIT': {
                'path': 'REFIT.h5'.format(),
                'buildings': {
                    BUILDING: {
                        'start_time': train_inicio,
                        'end_time': train_final
                }
                }
        }
    }
},
    'test': {
        'datasets': {
            'REFIT': {
                'path': 'REFIT.h5'.format(),
                'buildings': {
                     BUILDING: {
                         'start_time': test_inicio,
                         'end_time': test_final
                        } 
                    }
            }
    },   
    'metrics': ['mae'] #disaggregation performance with MAE
    }
} 
    
# Conduct experiment in NILMTK
api_results = API(experiment) #results are saved
exit()
    

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
import os, ast
from time import time
import nilmtk
import nilmtk_contrib
from nilmtk.api import API
from nilmtk.disaggregate import Mean, FHMMExact, CO, Hart85
from nilmtk_contrib.disaggregate import AFHMM, AFHMM_SAC, DSC, DAE, Seq2Point, Seq2Seq, RNN, WindowGRU
nilmtk.Appliance.allow_synonyms=False
import time
from datetime import datetime, timedelta
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
dataset = nilmtk.DataSet('REFIT.h5')

In [ ]:
#calculate normalized AE

# MAE*N = AE
# then, normalized AE = AE/sum(App)

App = next(dataset.buildings[5].elec[load].load(sample_period = 60))['power'].values.tolist()

inic_test_set = len(App)*0.33
App = [item for sublist in App for item in sublist]

api_results[0]/sum(App)

In [ ]:
#anr, wanr, mae and normalized AE obtained from the experimental evaluation 


wm_anr = [0.05, 0.07, 0.07, 0.02, 0.04, 0.01, 0.05, 0.03] #for the wm across 11 houses
wm_wanr = [0.001, 0.002, 0.002, 0, 0.001, 0, 0.004, 0]

nar = [0.71, 0.6, 0.62, 0.64, 0.66, 0.79, 0.70, 0.58] 

wm_normAE = [0.89, 0.64, 1.16, 0.51, 0.77, 1.28, 0.57, 0.71] 
wm_mae = [15.62, 19.98, 39.37, 3.07, 11.63, 4.85, 10.21, 4.85]

dw_anr = [0.17, 0.13, 0.11, 0.03, 0.12, 0.01, 0.05, 0.03]
dw_wanr = [0.006, 0.002, 0.002, 0, 0.002, 0, 0.002, 0]

dw_normAE = [0.69, 0.48, 0.42, 0.39, 0.36, 1.46, 2.51, 0.53]
dw_mae = [45.07, 24.07, 23.20, 3.61, 16.82, 5.72, 66.44, 4.27]

mw_anr = [0.01, 0.01, 0.05, 0.02, 0, 0.01, 0.12, 0.07]
mw_wanr = [ 0, 0, 0.003, 0.001, 0, 0, 0.064, 0.003]

mw_normAE = [0.76, 1.28, 0.63, 0.53, 1.36, 0.89, 0.31, 0.08]
mw_mae = [2.95, 3.89, 16.39, 3.74, 1.89, 2.82, 18.56, 2.07]

kt_anr = [0.06, 0.05, 0.04, 0.06, 0.07, 0.16, 0.03, 0.16]
kt_wanr = [0.013, 0.005, 0.004, 0.008, 0.009, 0.036, 0.005, 0.005]

kt_normAE = [0.24, 0.94, 0.44, 0.26, 0.31, 0.90, 0.87, 0.10]
kt_mae = [4.89, 18.27, 9.67, 4.75, 7.39, 143.46, 7.33, 2.48]


#compute pearson and spearman correlation coefficients
scipy.stats.pearsonr(kt_anr, kt_normAE)
scipy.stats.spearmanr(kt_anr, kt_normAE)

In [ ]:
#spider chart at the appliance-level

df = pd.DataFrame({
'Correlation': ['ANR, norm AE','WANR, norm AE','ANR, MAE','WANR, MAE'],
'WM': [0.02, 0.14, 0.90, 0.72],
 'DW': [0.21, 0.05, 0.66, 0.85], 
   'MW': [0.93, 0.56, 0.56, 0.52],
    'KT': [0.37, 0.07,0.12, 0.12]
})
     
# ------- PART 1: Define a function that do a plot for one line of the dataset!    
def make_spider( row, title, color):

    # number of variable
    categories=list(df)[1:]
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    ax = plt.subplot(2,2,row+1, polar=True, )

    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='black', size=20)

    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([0.25,0.5,0.75], ["0.25","0.5","0.75"], color="black", size=20)
    plt.ylim(0,1)

    # Ind1
    values=df.loc[row].drop('Correlation').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color=color, linewidth=2, linestyle='solid')
    ax.fill(angles, values, color=color, alpha=0.4)

    # Add a title
    plt.title(title, size=20, y=1.1)

    
# ------- PART 2: Apply the function to all individuals
# initialize the figure
my_dpi=70
plt.figure(figsize=(1200/my_dpi, 1200/my_dpi), dpi=my_dpi)
 
# Create a color palette:
my_palette = plt.cm.get_cmap("Set2", len(df.index))
 
# Loop to plot
for row in range(0, len(df.index)):
    #row = 3
    make_spider(row, title='Correlation '+df['Correlation'][row], color=my_palette(row))

In [ ]:
#spider chart at the house-level

df = pd.DataFrame({
'Correlation': ['ANR, norm AE','WANR, norm AE','ANR, MAE','WANR, MAE'],
'2': [0.60, 0.8, 0.8, 0.4],
 '3': [1, 0.32, 1, 0.32], 
   '5': [0.2, 0.11, 0.8, 0.95],
    '6': [0.95, 0.32, 0.63, 0.95], 
    '9': [0.8, 1, 0.8, 0.4],
    '11': [0.25, 0.26, 0.77, 0.77],
    '13': [0.63, 0.80, 0.63, 0.4], 
    '20': [1, 0.74, 1, 0.74]
})
 
# ------- PART 1: Define a function that do a plot for one line of the dataset!
 
def make_spider( row, title, color):

    # number of variable
    categories=list(df)[1:]
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    ax = plt.subplot(2,2,row+1, polar=True, )

    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='black', size=20)

    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([0.25,0.5,0.75], ["0.25","0.5","0.75"], color="black", size=20)
    plt.ylim(0,1)

    # Ind1
    values=df.loc[row].drop('Correlation').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, color=color, linewidth=2, linestyle='solid')
    ax.fill(angles, values, color=color, alpha=0.4)

    # Add a title
    plt.title(title, size=20, y=1.1)

    
# ------- PART 2: Apply the function to all individuals
# initialize the figure
my_dpi=70
plt.figure(figsize=(1200/my_dpi, 1200/my_dpi), dpi=my_dpi)
 
# Create a color palette:
my_palette = plt.cm.get_cmap("Set2", len(df.index))
 
# Loop to plot

for row in range(0, len(df.index)):
    make_spider( row=row, title='Correlation '+df['Correlation'][row], color=my_palette(row))